In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-09-05 16:04:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-05 16:04:59 (20.7 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [4]:
import tensorflow as tf

tf.__version__

'2.13.0'

In [5]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [6]:
import tensorflow as tf

In [7]:
# Get TensorFlow Datasets
import tensorflow_datasets as tfds

# Get all available datasets in TFDS
datasets_list = tfds.list_builders()

# Set our target dataset and see if it exists
target_dataset = "food101"
print(f"'{target_dataset}' in TensorFlow Datasets: {target_dataset in datasets_list}")

'food101' in TensorFlow Datasets: True


In [8]:
(train_data, test_data), ds_info = tfds.load(name="food101", # target dataset to get from TFDS
                                             split=["train", "validation"], # what splits of data should we get? note: not all datasets have train, valid, test
                                             shuffle_files=True, # shuffle files on download?
                                             as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                             with_info=True) # include dataset metadata? if so, tfds.load() returns tuple (data, ds_info)

In [9]:
ds_info

tfds.core.DatasetInfo(
    name='food101',
    full_name='food101/2.0.0',
    description="""
    This dataset consists of 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. On purpose, the training images were not cleaned, and thus still contain some amount of noise. This comes mostly in the form of intense colors and sometimes wrong labels. All images were rescaled to have a maximum side length of 512 pixels.
    """,
    homepage='https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/',
    data_path='/root/tensorflow_datasets/food101/2.0.0',
    file_format=tfrecord,
    download_size=4.65 GiB,
    dataset_size=4.77 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo n

In [10]:
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
})

In [11]:
class_names = ds_info.features['label'].names
class_names[:5]

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']

In [12]:
ds_info.features['image']

Image(shape=(None, None, 3), dtype=uint8)

In [13]:
def Resize(img, label, size=224):

  image = tf.image.resize(img, [size,size])
  return tf.cast(image, tf.float32), label

In [14]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=Resize, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(Resize, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [15]:
train_data, test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [16]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16") # set global policy to mixed precision

In [17]:
mixed_precision.global_policy() # should output "mixed_float16" (if your GPU is compatible with mixed precision)

<Policy "mixed_float16">

In [18]:
from tensorflow.keras import layers as L

def create_model():

  input_shape = (224, 224, 3)

  base_model = tf.keras.applications.EfficientNetB0(include_top=False)
  base_model.trainable = False

  Inputs = L.Input(shape = input_shape)

  x = base_model (Inputs, training=False)
  x = L.GlobalAveragePooling2D()(x)
  x = L.Dense(len(class_names))(x)
  Outputs = L.Activation("softmax", dtype=tf.float32)(x)

  model_1 = tf.keras.Model(Inputs, Outputs)
  return model_1

model_1 = create_model()
model_1.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 activation (Activation)     (None, 101)               0         
                                                                 
Total params: 4178952 (15.94 MB)
Trainable params: 129381 (50

In [19]:
# Create TensorBoard callback (already have "create_tensorboard_callback()" from a previous notebook)
from helper_functions import create_tensorboard_callback

# Create ModelCheckpoint callback to save model's progress
checkpoint_path = "model_checkpoints/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_accuracy", # save the model weights with best validation accuracy
                                                      save_best_only=True, # only save the best weights
                                                      save_weights_only=True, # only save model weights (not whole model)
                                                      verbose=0) # don't print out whether or not model is being saved

In [20]:
# Turn off all warnings except for errors
tf.get_logger().setLevel('ERROR')

'''# Fit the model with callbacks
history_101_food_classes_feature_extract = model_1.fit(train_data,
                                                     epochs=3,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback("training_logs",
                                                                                            "efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])'''

'# Fit the model with callbacks\nhistory_101_food_classes_feature_extract = model_1.fit(train_data, \n                                                     epochs=3,\n                                                     steps_per_epoch=len(train_data),\n                                                     validation_data=test_data,\n                                                     validation_steps=int(0.15 * len(test_data)),\n                                                     callbacks=[create_tensorboard_callback("training_logs", \n                                                                                            "efficientnetb0_101_classes_all_data_feature_extract"),\n                                                                model_checkpoint])'

In [21]:
'''results_feature_extract_model = model_1.evaluate(test_data)
results_feature_extract_model'''

'results_feature_extract_model = model_1.evaluate(test_data)\nresults_feature_extract_model'

In [22]:
'''created_model = create_model()
created_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

# Load the saved weights
created_model.load_weights(checkpoint_path)

# 4. Evaluate the model with loaded weights
results_created_model_with_loaded_weights = created_model.evaluate(test_data)'''

'created_model = create_model()\ncreated_model.compile(loss="sparse_categorical_crossentropy",\n                      optimizer=tf.keras.optimizers.Adam(),\n                      metrics=["accuracy"])\n\n# Load the saved weights\ncreated_model.load_weights(checkpoint_path)\n\n# 4. Evaluate the model with loaded weights\nresults_created_model_with_loaded_weights = created_model.evaluate(test_data)'

In [23]:
# import numpy as np
# assert np.isclose(results_feature_extract_model, results_created_model_with_loaded_weights).all()

In [24]:
# pip install -U tensorflow==2.13.0

In [25]:
# save_dir = "07_efficientnetb0_feature_extract_model_mixed_precision"
# model_1.save(save_dir)

In [26]:
# # Load model previously saved above
# loaded_saved_model = tf.keras.models.load_model(save_dir)

In [27]:
# # Check the layers in the base model and see what dtype policy they're using
# for layer in loaded_saved_model.layers[1].layers[:20]: # check only the first 20 layers to save output space
#     print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [28]:
for layer in model_1.layers:
    layer.trainable = True # set all layers to trainable

In [29]:
for layer in model_1.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling True float32 <Policy "mixed_float16">
normalization True float32 <Policy "mixed_float16">
rescaling_1 True float32 <Policy "mixed_float16">
stem_conv_pad True float32 <Policy "mixed_float16">
stem_conv True float32 <Policy "mixed_float16">
stem_bn True float32 <Policy "mixed_float16">
stem_activation True float32 <Policy "mixed_float16">
block1a_dwconv True float32 <Policy "mixed_float16">
block1a_bn True float32 <Policy "mixed_float16">
block1a_activation True float32 <Policy "mixed_float16">
block1a_se_squeeze True float32 <Policy "mixed_float16">
block1a_se_reshape True float32 <Policy "mixed_float16">
block1a_se_reduce True float32 <Policy "mixed_float16">
block1a_se_expand True float32 <Policy "mixed_float16">
block1a_se_excite True float32 <Policy "mixed_float16">
block1a_project_conv True float32 <Policy "mixed_float16">
block1a_project_bn True float32 <Policy "mixed_float16">
block2a_expand_conv True float32 <Policy "mixed_float

In [30]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [31]:
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

In [32]:
model_1.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                        metrics=["accuracy"])

In [33]:
# pip install -U tensorflow==2.13.0

In [34]:
history_101_food_classes_all_data_fine_tune = model_1.fit(train_data,
                                                        epochs=100, # fine-tune for a maximum of 100 epochs
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # track the model training logs
                                                                   model_checkpoint, # save only the best model during training
                                                                   early_stopping, # stop model after X epochs of no improvements
                                                                   reduce_lr]) # reduce the learning rate after X epochs of no improvement

Saving TensorBoard log files to: training_logs/efficientb0_101_classes_all_data_fine_tuning/20230905-160509
Epoch 1/100
2368/2368 [==============================] - 464s 171ms/step - loss: 1.6037 - accuracy: 0.5931 - val_loss: 0.8820 - val_accuracy: 0.7500 - lr: 1.0000e-04
Epoch 2/100
2368/2368 [==============================] - 409s 171ms/step - loss: 0.8419 - accuracy: 0.7704 - val_loss: 0.7362 - val_accuracy: 0.7905 - lr: 1.0000e-04
Epoch 3/100
2368/2368 [==============================] - 372s 156ms/step - loss: 0.5336 - accuracy: 0.8500 - val_loss: 0.7519 - val_accuracy: 0.8006 - lr: 1.0000e-04
Epoch 4/100
2368/2368 [==============================] - ETA: 0s - loss: 0.3086 - accuracy: 0.9085
Epoch 4: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
2368/2368 [==============================] - 374s 157ms/step - loss: 0.3086 - accuracy: 0.9085 - val_loss: 0.7913 - val_accuracy: 0.8006 - lr: 1.0000e-04
Epoch 5/100
2368/2368 [==============================] - 376s 15